In [0]:
import numpy as np
import cv2
import scipy as sp
import skimage as si
from matplotlib import pyplot as plt
%matplotlib inline

# Loading an image
load an image  - using OpenCV

In [0]:
!wget https://cms-assets.tutsplus.com/uploads/users/1181/posts/30095/image/boat850x850.jpg

img = cv2.imread('boat850x850.jpg')

plt.imshow(img)


# Q1
Q1.a

In [0]:
img = cv2.imread('boat850x850.jpg')

edges = cv2.Canny(img,25,255)

plt.subplot(121)
plt.imshow(img, cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])

plt.subplot(122)
plt.imshow(edges, cmap = 'gray')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])

plt.show()

Q1.b

In [0]:
img = cv2.imread('boat850x850.jpg')

gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
gray = np.float32(gray)
dst = cv2.cornerHarris(gray,2,3,0.04)

#result is dilated for marking the corners
dst = cv2.dilate(dst,None)

# Threshold for an optimal value
img[dst>0.01*dst.max()]=[0,0,255]

plt.imshow(img)

Q1.c

In [0]:
img = cv2.imread('boat850x850.jpg')

gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) 

sift = cv2.xfeatures2d.SIFT_create()
#finding the key points without mask
kp = sift.detect(gray,None)

#drawing the key points on the image
img=cv2.drawKeypoints(gray,kp) 

# cv2.imwrite('sift_keypoints.jpg',img)

Q1.d

In [0]:
!wget https://pixwords.solutions/assets/images/image-38f58bcb330564cf15381fa68a74f831-STATUE%20OF%20LIBERTY.JPG
!wget http://cafe.themarker.com/media/t/336/639/8/file_0.jpg

img1 = cv2.imread('image-38f58bcb330564cf15381fa68a74f831-STATUE%20OF%20LIBERTY.JPG',0)          # queryImage
img2 = cv2.imread('file_0.jpg',0) # trainImage


# Initiate SIFT detector
sift = cv2.xfeatures2d.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

# BFMatcher with default params
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1,des2, k=2)

# Apply ratio test
good = []
for m,n in matches:
    if m.distance < 0.75*n.distance:
        good.append([m])
        
# cv2.drawMatchesKnn expects list of lists as matches.
img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,good,flags=2)

plt.imshow(img3)
plt.show()

 Q1.e

In [0]:
img = cv2.imread('boat850x850.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) 
  
# Apply edge detection method on the image 
edges = cv2.Canny(gray,50,150,apertureSize = 3) 
  
# This returns an array of r and theta values 
lines = cv2.HoughLines(edges,1,np.pi/180, 200) 
  
# The below for loop runs till r and theta values are in the range of the 2d array 
for r,theta in lines[0]:       
    # Stores the value of cos(theta) in a 
    a = np.cos(theta) 
    # Stores the value of sin(theta) in b 
    b = np.sin(theta) 
    # x0 stores the value rcos(theta) 
    x0 = a*r 
    # y0 stores the value rsin(theta) 
    y0 = b*r 
    # x1 stores the rounded off value of (rcos(theta)-1000sin(theta)) 
    x1 = int(x0 + 1000*(-b)) 
    # y1 stores the rounded off value of (rsin(theta)+1000cos(theta)) 
    y1 = int(y0 + 1000*(a)) 
    # x2 stores the rounded off value of (rcos(theta)+1000sin(theta)) 
    x2 = int(x0 - 1000*(-b)) 
    # y2 stores the rounded off value of (rsin(theta)-1000cos(theta)) 
    y2 = int(y0 - 1000*(a)) 
      
    # cv2.line draws a line in img from the point(x1,y1) to (x2,y2). 
    cv2.line(img,(x1,y1), (x2,y2), (0,0,255),2) 
    plt.imshow(img)
      
plt.imshow(img)

#Q2

we send to k-means function the data (Z=8 random points), K = number of means (3), criteria to stop the algorithm (explanation below), number of attempts (10).

the k-means returns the k centers (means) according to the flag KMEANS_RANDOM_CENTERS and the points with the labels of the group it's belong.

criteria : stop the algorithm when the algorithm iteration if specified accuracy, epsilon, is reached (in this case epsilon = 0.1) or after the specified number of iterations (in this case max_iter = 10).


In [0]:
randArray = np.random.randint (1,13,(8,2))
print (randArray)

# convert to np.float32
Z = np.float32(randArray)

# define criteria and apply kmeans()
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 50, 1.0)
ret,label,center=cv2.kmeans(Z,3,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)

# Now separate the data, Note the flatten()
A = Z[label.ravel()==0]
B = Z[label.ravel()==1]
C = Z[label.ravel()==2]

# Plot the data
plt.scatter(A[:,0],A[:,1],c = 'b')
plt.scatter(B[:,0],B[:,1],c = 'r')
plt.scatter(C[:,0],C[:,1],c = 'g')
plt.scatter(center[:,0],center[:,1],s = 80,c = 'y', marker = 's')
plt.xlabel('X'),plt.ylabel('Y')

plt.show()

#Q3

calculate B (=a,b,c) by the formulla X(t)*X*b = X(t)*Y

In [0]:
arr = [(1,3.96),(4,27.96),(3,15.15),(5,45.8),(2,7.07),(6,69.4)]

#create X (x^2=p[0]*p[0])
X = np.array(list(map(lambda p: ([p[0] * p[0], p[0], 1]), arr)))

#create Y
Y = np.array(list(map(lambda p: p[1], arr)))

#create B = (X(t)*X)(-1) * X(t) * Y(t)
XtX_1 = np.linalg.inv(np.dot(X.T, X))
B = np.dot(np.dot(XtX_1, X.T), Y.T)

print('X= ', X)
print('Y= ', Y)
print('B= ', B)

x = np.linspace(-4,6,50)
y = B[0]*x*x + B[1]*x + B[2]

# Plot the data
plt.scatter(np.array(arr)[:,0] , np.array(arr)[:,1], c = 'b')
plt.scatter(x, y, s=1, c = 'r')
plt.xlabel('X'),plt.ylabel('Y')

plt.show()






#Q4

calculate covariance matrix:
[sum of (Xi - meanX)^2                          sum of (Xi - meanX)(Yi-meanY)]
[sum of (Xi - meanX)(Yi-meanY)                          sum of (Xi - meanX)^2]

calculate eigenvalues:
det(cov_matrix - eigenval*I) = 0

calculate eigenvectors:
cov_matrix * [eigenvec] = [eigenvalue*eigenvector]

W = [eigenvector1    eigenvector2]

PCA matrix: W(t) * (array - mean_vector)

RESTORED matrix: W(t) * pca_matrix + mean_vector

In [0]:
arr0 = [(2.5,2.9), (0.5,1.2), (2.2,3.4), (1.9,2.7), (3.1,3.5), (2.3,3.2), (2,2.1), (1,1.6), (1.5,2.1), (1.1,1.4)]
arr = np.array(arr0).T
print ('ARRAY: '), print (arr)

mean_vec = np.array([np.mean(arr0, axis=0)]).T
print (mean_vec)

#calculate covariance matrix of features
cov_matrix = np.cov(arr)
print ('COVARIANCE MATRIX: '), print (cov_matrix)

#calculate eigenvalues
e_vals, e_vecs = sp.linalg.eig(cov_matrix)
print ('EIGENVALUES: '), print (e_vals)
print ('EIGENVECTORS (W): '), print (e_vecs)

#calculate PCA matrix
pca_matrix = np.array([np.dot(e_vecs.T[1], arr - mean_vec)])
print ('PCA MATRIX: '), print (pca_matrix)

#calculate restored matrix
res_matrix = (np.dot(np.array([e_vecs[0]]).T, pca_matrix) + mean_vec).T
print ('RESTORED MATRIX: '), print (res_matrix)

# Plot the data

#original data
plt.scatter(np.array(arr0)[:,0] , np.array(arr0)[:,1], c = 'b')
#data after PCA
# plt.scatter(np.array(pca_matrix)[:,0] , 0, c = 'r')
plt.plot(pca_matrix, np.zeros((1,10)), '*', c = 'r')
#restored data
plt.scatter(np.array(res_matrix)[:,0] , np.array(res_matrix)[:,1], c = 'g')
plt.xlabel('X'),plt.ylabel('Y')

plt.show()
